<a id='top'></a>

# My First xG Model
##### Notebook to create a basic xG model from a sample dataset of just under 11,000 shots

### By [Edd Webster](https://www.twitter.com/eddwebster)
Notebook first written: 15/06/2021<br>
Notebook last updated: 15/06/2021

![title](../../../img/expected_goals_visual.png)

Photo credit to David Sumpter ([@Soccermatics](https://twitter.com/Soccermatics)).

## 1. Notebook Setup

### Import Libraries and Modules

In [1]:
# Python ≥3.5 (ideally)
import sys, getopt
assert sys.version_info >= (3, 5)
import csv

# Import Dependencies
%matplotlib inline

# Math Operations
import numpy as np
import math
from math import pi

# Datetime
import datetime
from datetime import date
import time

# Data Preprocessing
import pandas as pd
import pandas_profiling as pp
import os
import re
import random
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os
from os.path import basename

# Working with JSON
import json
from pandas.io.json import json_normalize

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib.patches import Arc
from matplotlib.colors import ListedColormap
import plotly
import plotly.graph_objects as go
import ruamel.yaml
import seaborn as sns
import missingno as msno


# Machine Learning
import scipy as sp
import scipy.spatial
from scipy.spatial import distance
from sklearn.ensemble import RandomForestClassifier
#from sklearn.inspection import permutation_importance
import sklearn.metrics as sk_metrics
from sklearn.metrics import log_loss, brier_score_loss, roc_auc_score , roc_curve, average_precision_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from scikitplot.metrics import plot_roc_curve, plot_precision_recall_curve, plot_calibration_curve

# Display in Jupyter
from IPython.display import Image, Video, YouTubeVideo
from IPython.core.display import HTML

# Ignore Warnings
import warnings
warnings.filterwarnings(action='ignore', message='^internal gelsd')

print('Setup Complete')

Setup Complete


### Notebook Settings

In [2]:
# Display all columns of pandas DataFrames
pd.set_option('display.max_columns', None)

---

## 2. Data Sources

In [3]:
# Read in raw Shots CSV as a pandas DataFrame
data_dir_shots = os.path.join('..', '..', '..', 'data', 'shots')
df_shots = pd.read_csv(os.path.join(data_dir_shots, 'raw', 'ShotData.csv'))

In [4]:
# Display the first five rows of the raw DataFrame, df_shots
df_shots.head()

,match_minute,match_second,position_x,position_y,play_type,BodyPart,Number_Intervening_Opponents,Number_Intervening_Teammates,Interference_on_Shooter,outcome
0,7,24,3.24,-0.75,Open Play,Head,1,0,Low,Goal
1,34,55,24.94,0.75,Open Play,Left,3,0,Medium,Missed
2,60,33,3.74,-0.50,Open Play,Right,1,0,Low,Missed
3,68,42,21.45,-8.48,Direct freekick,Left,4,2,Low,Blocked
4,18,0,10.23,4.99,Open Play,Head,1,0,Low,Missed


In [5]:
# Print the shape of the raw DataFrame, df_shots
print(df_shots.shape)

(10925, 10)


In [6]:
# Print the column names of the raw DataFrame, df_shots
print(df_shots.columns)

Index(['match_minute', 'match_second', 'position_x', 'position_y', 'play_type',
       'BodyPart', 'Number_Intervening_Opponents',
       'Number_Intervening_Teammates', 'Interference_on_Shooter', 'outcome'],
      dtype='object')


---

## 3. Data Engineering

### Convert Pitch Dimensions to Standard Coordinates

In [7]:
## Define 'standardised' pitch x and y lengths
pitch_x_start = -53
pitch_x_end = 53
pitch_y_start = -34
pitch_y_end = 34
pitch_length_x = pitch_x_end - pitch_x_start 
pitch_length_y = pitch_y_end - pitch_y_start 

In [8]:
# Convert to standard pitch dimensions

## Convert default coordinates of x(106, 0) and y(-33.92, 33.92) to a standardised range of x(-53, +53) and y(-34, +34) - the average pitch size
df_shots['position_xM'] = (pitch_length_x/2)-(df_shots['position_x'])
df_shots['position_yM'] = (df_shots['position_y'] / df_shots['position_y'].max()) * (pitch_length_y/2)

## Create reverse columns of standardised xM' and 'yM' columns (used later for visualisation purposes only)
df_shots['position_xM_r'] = -df_shots['position_xM']
df_shots['position_yM_r'] = -df_shots['position_yM']

## Create standardised columns where the start is 0 (used for visualisations but not for ML)
df_shots['position_xM_std'] = df_shots['position_xM'] + (pitch_length_x/2)
df_shots['position_yM_std'] = df_shots['position_yM'] + (pitch_length_y/2)

## Create reverse columns for standardised columns (used later for visualisation purposes only)
df_shots['position_xM_std_r'] = df_shots['position_xM_std'] + (pitch_length_x/2)
df_shots['position_yM_std_r'] = df_shots['position_yM_std'] + (pitch_length_y/2)

### Clean Attributes

In [9]:
# There are two values for 'Direct Corner' - 'Direct Corner and 'Direct corner'. Replace values to all be the same.
df_shots['play_type'] = df_shots['play_type'].replace('Direct Corner', 'Direct corner', regex=True)

In [10]:
# Shot play types and their frequency
df_shots.groupby(['BodyPart']).play_type.count()

BodyPart
Head     1862
Left     3548
Other      70
Right    5445
Name: play_type, dtype: int64

In [11]:
# 70 Open Play shots have a `BodyPart` value of 'Unknown'. These are left for now but dealt with the the Univariate Analysis section.

## Define dictionary of BodyPart codes
dict_bodypart_codes = {'Left': 1,
                       'Right': 2,
                       'Head': 3,
                       'Other': 4
                      }

## Map BodyPartCode to DataFrame
df_shots['BodyPartCode'] = df_shots['BodyPart'].map(dict_bodypart_codes)

##### `Interference_on_Shooter`
In the initial data handling, we saw that there were 43 missing values in the `Interference_on_Shooter` column.

To prevent further issues later Feature Engineering, all NULL valus are replaced with a string equal to 'Unknown'.

In [12]:
df_shots['Interference_on_Shooter'].unique()

array(['Low', 'Medium', 'High', nan], dtype=object)

In [13]:
# Replace null values with 'Unknown' string
df_shots['Interference_on_Shooter'] = df_shots['Interference_on_Shooter'].replace(np.nan, 'Unknown', regex=True)

In [14]:
# Shot play types and their frequency
df_shots.groupby(['Interference_on_Shooter']).play_type.count()

Interference_on_Shooter
High       1694
Low        4050
Medium     5138
Unknown      43
Name: play_type, dtype: int64

Create attribute `Interference_on_Shooter_code`

In [15]:
# Define dictionary of Interference_on_Shooter codes
dict_inferenceshooter_codes = {'Low': 1,
                               'Medium': 2,
                               'High': 3,
                               'Unknown': 4
                              }

# Map Interference_on_Shooter_Code to DataFrame
df_shots['Interference_on_Shooter_Code'] = df_shots['Interference_on_Shooter'].map(dict_inferenceshooter_codes)

### Create New Features

In [16]:
# Create new features - distance_to_goalM, 'distance_to_centerM', and 'angle'
df_shots['distance_to_goalM'] = np.sqrt((((pitch_length_x/2) - df_shots['position_xM'])**2) + ((df_shots['position_yM'])**2))
df_shots['distance_to_centerM'] = np.abs(df_shots['position_yM'])
df_shots['angle'] = np.absolute(np.degrees(np.arctan((df_shots['position_yM']) / ((pitch_length_x/2) - df_shots['position_xM']))))

### Create Target

##### `isGoal`
Boolean field, used as the Target in the Machine Learning i.e. whether the shot resulted in a goal (1) or not (0).

In [17]:
# Shot outcomes types and their frequency
df_shots.groupby(['outcome']).outcome.count()

outcome
Blocked      2155
Goal         1331
GoalFrame     215
Missed       4254
Saved        2927
owngoal        43
Name: outcome, dtype: int64

In [18]:
# Define dictionary of outcomes
dict_outcome_codes = {'Blocked': 0,
                      'Goal': 1,
                      'GoalFrame': 0,
                      'Missed': 0,
                      'Saved': 0,
                      'owngoal': 1
                     }

# Map isGoal Boolean to DataFrame
df_shots['isGoal'] = df_shots['outcome'].map(dict_outcome_codes)

### Remove Own Goals

In [19]:
# Filter out Own Goals from the dataset
df_shots = df_shots[df_shots['outcome'] != 'owngoal']

### Create Open Play (OP) Shots DataFrame

In [20]:
# Subset DataFrame for non-penalty shots
df_op_shots = df_shots[(df_shots['play_type'] == 'Open Play')].copy()

---

## 5. Data Visualisation

In [ ]:
# CODE HERE

---

## 6. Expected Goals Modeling

In [21]:
df_op_shots.head()

,match_minute,match_second,position_x,position_y,play_type,BodyPart,Number_Intervening_Opponents,Number_Intervening_Teammates,Interference_on_Shooter,outcome,position_xM,position_yM,position_xM_r,position_yM_r,position_xM_std,position_yM_std,position_xM_std_r,position_yM_std_r,BodyPartCode,Interference_on_Shooter_Code,distance_to_goalM,distance_to_centerM,angle,isGoal
0,7,24,3.24,-0.75,Open Play,Head,1,0,Low,Goal,49.76,-0.751769,-49.76,0.751769,102.76,33.248231,155.76,67.248231,3,1,3.326072,0.751769,13.063042,1
1,34,55,24.94,0.75,Open Play,Left,3,0,Medium,Missed,28.06,0.751769,-28.06,-0.751769,81.06,34.751769,134.06,68.751769,1,2,24.951328,0.751769,1.726550,0
2,60,33,3.74,-0.50,Open Play,Right,1,0,Low,Missed,49.26,-0.501179,-49.26,0.501179,102.26,33.498821,155.26,67.498821,2,1,3.773431,0.501179,7.632460,0
4,18,0,10.23,4.99,Open Play,Head,1,0,Low,Missed,42.77,5.001769,-42.77,-5.001769,95.77,39.001769,148.77,73.001769,3,1,11.387300,5.001769,26.055464,0
5,31,26,24.19,0.75,Open Play,Right,3,1,Medium,Missed,28.81,0.751769,-28.81,-0.751769,81.81,34.751769,134.81,68.751769,2,2,24.201679,0.751769,1.780046,0


In [22]:
# Select Features of interest
feature_cols = ['distance_to_goalM', 
                'angle', 
                'BodyPartCode'
               ]

# Define Target
target_col = ['isGoal']

# Assign Feature and Target to separate DataFrames and Series
X = df_op_shots[feature_cols]
y = df_op_shots[target_col]

In [23]:
# Split the dataset into a train set and a test set - 70:30 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [24]:
# Train Logistic Regression model
reg_initial = LogisticRegression(random_state=42)
reg_initial.fit(X_train, np.array(y_train).ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
# Predict on the Test set
pred_probs = reg_initial.predict_proba(X_test)[:,1]
pred_vals = reg_initial.predict(X_test)

In [27]:
# Compute performance of the model

## Accuracy
print(f'Accuracy of initial model: {sk_metrics.accuracy_score(y_test, pred_vals)*100:.1f}%')

## Log Loss
pred_probs = reg_initial.predict_proba(X_test)[:,1]
print(f'Log Loss of initial model: {sk_metrics.log_loss(y_test, pred_probs):.5f}')

## AUC
print(f'AUC of our initial model: {sk_metrics.roc_auc_score(y_test, pred_probs)*100:.2f}%')

Accuracy of initial model: 88.6%
Log Loss of initial model: 0.31312
AUC of our initial model: 75.41%


In [28]:
# Logistic Model coefficients
for i, col in enumerate(X_train.columns):
    print(f"Coefficient of {col}: {reg_initial.coef_[0][i]:.3f}")

Coefficient of distance_to_goalM: -0.165
Coefficient of angle: -0.008
Coefficient of BodyPartCode: -0.511


#### Predictions
There are two types of classification predictions that can be made with the finalised model:
*    class predictions; and
*    probability predictions.

##### Single Class Prediction
Single class predictions predict which category the instant of data belongs to. As even good quality chances are rarely greater than 30-40% likely to be goals, this type of prediction is not much use in this scenario, but has been included below for reference.

In [ ]:
xNew = df_metrica_trans[features]

In [ ]:
yNew = reg_metrica.predict(xNew)

In [ ]:
yNew

##### Probability Predictions
Probability predictions determines the probability of each instance belonging to the first and second classes (0 and 1), as a percentage i.e. Expected Goals (xG).

In [ ]:
yNew = reg_metrica.predict_proba(xNew)

In [ ]:
yNew

In [ ]:
# Convert the Probability Predictions array to a pandas DataFrame
df_metrica_predictions = pd.DataFrame(yNew, columns = ['prob_no_goal', 'prob_goal'])

In [ ]:
df_metrica_predictions.head()

In [ ]:
# Join the Probability Predictions back onto the original Metrica shots DataFrame
df_metrica_xg = pd.merge(df_metrica, df_metrica_predictions, left_index=True, right_index=True)

# Display DataFrame
df_metrica_xg.head()

---

***Visit my website [eddwebster.com](https://www.eddwebster.com) or my [GitHub Repository](https://github.com/eddwebster) for more projects. If you'd like to get in contact, my Twitter handle is [@eddwebster](http://www.twitter.com/eddwebster) and my email is: edd.j.webster@gmail.com.***

[Back to the top](#top)